# Accessing Spotify API Data

In order to further explore my listening data, I wanted to access the audio features of each track that I had listened to in the past year. To do so, I needed to format a GET request to the Spotify Web API. By using a Python library entitled "Spotipy" that works directly with the web API, I was able to obtain the audio features for my top 1500 songs from the past year.

In [1]:
import pandas as pd

In [4]:
pd.read_csv("features/features.csv", index_col=0)

,title,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Callaita,0.610,0.624,2.0,-4.773,1.0,0.3090,0.600000,0.000002,0.2430,0.2440,176.169,audio_features,2TH65lNHgvLxCKXM3apjxI,spotify:track:2TH65lNHgvLxCKXM3apjxI,https://api.spotify.com/v1/tracks/2TH65lNHgvLx...,https://api.spotify.com/v1/audio-analysis/2TH6...,250534.0,4.0
1,Sacrifices (with EARTHGANG & J. Cole feat. Smi...,0.825,0.672,0.0,-7.108,1.0,0.4180,0.085800,0.000000,0.1230,0.2460,147.010,audio_features,7wTA0NKIm6T7nP2kaymU2a,spotify:track:7wTA0NKIm6T7nP2kaymU2a,https://api.spotify.com/v1/tracks/7wTA0NKIm6T7...,https://api.spotify.com/v1/audio-analysis/7wTA...,382307.0,4.0
2,Envy Me,0.740,0.488,1.0,-7.664,0.0,0.2700,0.234000,0.000000,0.2410,0.5840,149.042,audio_features,7rvyVWja33WG9R97oeJAjx,spotify:track:7rvyVWja33WG9R97oeJAjx,https://api.spotify.com/v1/tracks/7rvyVWja33WG...,https://api.spotify.com/v1/audio-analysis/7rvy...,120133.0,4.0
3,Me Odias?,0.786,0.581,1.0,-6.387,0.0,0.1970,0.552000,0.000000,0.0859,0.3300,131.874,audio_features,5I31UVARw4Di9SooJGN9el,spotify:track:5I31UVARw4Di9SooJGN9el,https://api.spotify.com/v1/tracks/5I31UVARw4Di...,https://api.spotify.com/v1/audio-analysis/5I31...,234550.0,4.0
4,Quiero,0.651,0.737,9.0,-7.416,1.0,0.0415,0.044700,0.000000,0.1560,0.7030,104.832,audio_features,0iEW2LlHNXVyZhwO96sL3z,spotify:track:0iEW2LlHNXVyZhwO96sL3z,https://api.spotify.com/v1/tracks/0iEW2LlHNXVy...,https://api.spotify.com/v1/audio-analysis/0iEW...,209583.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,Shots,0.829,0.582,7.0,-5.721,0.0,0.0415,0.525000,0.000000,0.0759,0.5100,108.028,audio_features,6eSU1exzDvBEXmBRxspeh5,spotify:track:6eSU1exzDvBEXmBRxspeh5,https://api.spotify.com/v1/tracks/6eSU1exzDvBE...,https://api.spotify.com/v1/audio-analysis/6eSU...,109373.0,4.0
1496,Peaceful Forest,0.211,0.425,7.0,-46.448,0.0,0.0542,0.880000,0.613000,0.1840,0.0122,116.884,audio_features,28ilcqErfNjitJ7T3byHoX,spotify:track:28ilcqErfNjitJ7T3byHoX,https://api.spotify.com/v1/tracks/28ilcqErfNji...,https://api.spotify.com/v1/audio-analysis/28il...,181204.0,4.0
1497,It's A Vibe,0.822,0.502,7.0,-7.380,1.0,0.1480,0.031200,0.000887,0.1140,0.5250,73.003,audio_features,6H0AwSQ20mo62jGlPGB8S6,spotify:track:6H0AwSQ20mo62jGlPGB8S6,https://api.spotify.com/v1/tracks/6H0AwSQ20mo6...,https://api.spotify.com/v1/audio-analysis/6H0A...,210200.0,4.0
1498,Peace of Mind,0.421,0.677,4.0,-6.183,1.0,0.0343,0.000989,0.000097,0.2110,0.6830,127.670,audio_features,1GqlvSEtMx5xbGptxOTTyk,spotify:track:1GqlvSEtMx5xbGptxOTTyk,https://api.spotify.com/v1/tracks/1GqlvSEtMx5x...,https://api.spotify.com/v1/audio-analysis/1Gql...,303587.0,4.0


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

import numpy as np
import pandas as pd
import seaborn as sns
import json
import time
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

In [8]:
CLIENT_USERNAME = "willfurtado"
SPOTIPY_CLIENT_ID = "0c58b8f377294e1393b6ff20d1db34fc"
SPOTIPY_CLIENT_SECRET = "12fb3865a39343aba75ec4b118f6adf9"
SPOTIPY_REDIRECT_URI = "https://localhost:8888"

PODCAST_ARTISTS = ['VIEWS with David Dobrik and Jason Nash', 'The California Golden Bearcast', 
                 'Whiskey Ginger w/ Andrew Santino', 'The Tiny Meat Gang Podcast',
                 'Stuff You Should Know','Patriots Unfiltered','Cal Rivals Excellent Podcast Experience',
                 'Curious with Josh Peck','Locked On Patriots - Daily Podcast On The New England Patriots',
                 'Skotcast with Jeff Wittek & Scotty Sire','Anything Goes with Emma Chamberlain',
                 'Call Her Daddy', 'Office Ladies', 'That Made All the Difference','Pardon My Take', 
                  'My Favorite Theorem', 'The James Altucher Show', 'Zane and Heath: Unfiltered',
                   'With Authority','The Numberphile Podcast', 'Billionaires Getting Interviewed',
                  'Elon Musk Interviews','Cover 3 College Football Podcast']

WHITE_NOISE = ['Nature Sounds', 'Sounds Of Nature : Thunderstorm, Rain','Calmsound']

DIFFICULT_SONGS = {'I Know (feat. Mick Jenkins)': 'I Know Mick Jenkins', 
                   'Take Me Home, Country Roads - Rerecorded': 'Take Me Home Country Roads',
                  'Chica Paranormal - Verdun Remix': 'Chica Paranormal'}

UNSEARCHABLE_SONGS = {'!!!!!!!':'0rQtoQXQfwpDW0c7Fw1NeM'}

In [9]:
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, 
    client_id=SPOTIPY_CLIENT_ID, 
    client_secret=SPOTIPY_CLIENT_SECRET,
    username='willfurtado',
    redirect_uri=SPOTIPY_REDIRECT_URI))

In [11]:
with open('personal_data/StreamingHistory0.json') as file:
    data = json.load(file)
df0 = pd.DataFrame(data)

with open('personal_data/StreamingHistory1.json') as file:
    data1 = json.load(file)
df1 = pd.DataFrame(data1)

df = df0.append(df1, ignore_index=True)

df['secPlayed'] = round(df['msPlayed'] / 1000, 1)
df = df.drop(columns=['msPlayed'])
STRTIME_FORMAT = '%Y-%m-%d %H:%M'
df['endTime'] = pd.to_datetime(df['endTime'], format=STRTIME_FORMAT)

music = df[~df['artistName'].isin(PODCAST_ARTISTS + WHITE_NOISE)].reset_index(drop=True)
podcasts = df[df['artistName'].isin(PODCAST_ARTISTS)].reset_index(drop=True)

In [12]:
def get_spotify_uri(song):
    """Returns the corresponding spotify URI from a given song title"""
    if song in DIFFICULT_SONGS:
        search_results = sp.search(q=DIFFICULT_SONGS[song], type='track', limit=1)
    elif song in UNSEARCHABLE_SONGS:
        return UNSEARCHABLE_SONGS[song]
    else:
        search_results = sp.search(q=song, type='track', limit=1)
    try:
        return search_results['tracks']['items'][0]['id']
    except (AttributeError, IndexError) as err:
        print('No results for {}'.format(song))
        pass
            
unique_songs = music.groupby('trackName').count().sort_values('secPlayed', ascending=False).index

column_labels = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 
                 'analysis_url', 'duration_ms', 'time_signature']

features_df = pd.DataFrame(columns=column_labels)

In [1]:
for song in unique_songs:
    uri = get_spotify_uri(song)
    features = sp.audio_features([uri])[0]
    dataframe = pd.DataFrame(data=features, index=[song])
    features_df = features_df.append(dataframe, sort=False)

NameError: name 'unique_songs' is not defined

In [49]:
features_df.to_csv('./features/features.csv', index=True)